In [1]:
import setup

In [2]:
import numpy as np
import cv2
from detector import FaceDetector

In [5]:
detector = FaceDetector(
    "../checkpoints/yolo_nano_v1/yolo_nano_v1.onnx", conf_thre=0.1, device="cuda"
)

*************** EP Error ***************
EP Error /onnxruntime_src/onnxruntime/core/providers/cuda/cuda_call.cc:121 std::conditional_t<THRW, void, onnxruntime::common::Status> onnxruntime::CudaCall(ERRTYPE, const char*, const char*, ERRTYPE, const char*, const char*, int) [with ERRTYPE = cudaError; bool THRW = true; std::conditional_t<THRW, void, onnxruntime::common::Status> = void] /onnxruntime_src/onnxruntime/core/providers/cuda/cuda_call.cc:114 std::conditional_t<THRW, void, onnxruntime::common::Status> onnxruntime::CudaCall(ERRTYPE, const char*, const char*, ERRTYPE, const char*, const char*, int) [with ERRTYPE = cudaError; bool THRW = true; std::conditional_t<THRW, void, onnxruntime::common::Status> = void] CUDA failure 999: unknown error ; GPU=32593 ; hostname=qninhdt-pc ; file=/onnxruntime_src/onnxruntime/core/providers/cuda/cuda_execution_provider.cc ; line=245 ; expr=cudaSetDevice(info_.device_id); 

 when using ['CUDAExecutionProvider']
Falling back to ['CUDAExecutionProvider

RuntimeError: /onnxruntime_src/onnxruntime/core/providers/cuda/cuda_call.cc:121 std::conditional_t<THRW, void, onnxruntime::common::Status> onnxruntime::CudaCall(ERRTYPE, const char*, const char*, ERRTYPE, const char*, const char*, int) [with ERRTYPE = cudaError; bool THRW = true; std::conditional_t<THRW, void, onnxruntime::common::Status> = void] /onnxruntime_src/onnxruntime/core/providers/cuda/cuda_call.cc:114 std::conditional_t<THRW, void, onnxruntime::common::Status> onnxruntime::CudaCall(ERRTYPE, const char*, const char*, ERRTYPE, const char*, const char*, int) [with ERRTYPE = cudaError; bool THRW = true; std::conditional_t<THRW, void, onnxruntime::common::Status> = void] CUDA failure 999: unknown error ; GPU=32593 ; hostname=qninhdt-pc ; file=/onnxruntime_src/onnxruntime/core/providers/cuda/cuda_execution_provider.cc ; line=245 ; expr=cudaSetDevice(info_.device_id); 



In [ ]:
img = cv2.imread("../inferences/1479831.jpg")

In [ ]:
detected_faces = detector.detect(img)
detected_faces

NameError: name 'detector' is not defined

In [ ]:
from model.evaluators.postprocess import postprocess

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import ImageDraw
from PIL import ImageFont
from torchvision.transforms.functional import to_pil_image

def unnormailze(image):
    image = image.cpu().numpy()
    image = image.transpose(1, 2, 0)
    image = image * [0.2272, 0.2248, 0.2345]
    image = image + [0.6596, 0.6235, 0.5875]
    image = image * 255
    image = image.astype('uint8')
    return image

def get_sample_image(image, sample):
    # convert torch tensor to PIL image
    image = to_pil_image(unnormailze(image.cpu()))

    draw = ImageDraw.Draw(image)

    for i in range(sample['boxes'].shape[0]):
        box = sample['boxes'][i]
        x, y, w, h = int(box[0]) - int(box[2]) / 2, int(box[1]) - int(box[3]) / 2, int(box[2]), int(box[3])

        color = 'red' if sample['gender'][i] == 0 else 'blue'

        draw.rectangle(((x, y), (x+w, y+h)), outline=color, width=3)

    return image

In [ ]:
def visualize_sample(image, target, pred):
    # draw in 1x2 grid
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

    ax1.imshow(get_sample_image(image, target))
    ax1.set_title('Ground Truth')
    ax1.axis('off')

    ax2.imshow(get_sample_image(image, pred))
    ax2.set_title('Prediction')
    ax2.axis('off')

    plt.show()

In [ ]:
image, target = next(loader)

pred = model(image)[0]
pred = postprocess(pred, 0.2, 0.5)

print(target)

for i in range(len(pred)):
    visualize_sample(image[i], target[i], pred[i])


NameError: name 'loader' is not defined